In [5]:
from sqlalchemy import create_engine
from datetime import timedelta, date
from datetime import datetime
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import pymysql
import pickle
import csv
import os
import time

In [2]:
# tf gpu 메모리 관련 코드 
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [6]:
today = str(date.today() - timedelta(days=1)).replace('-','')
yesterday = str(date.today() - timedelta(days=2)).replace('-','')
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn

#DB 내 존재하는 테이블(종목) 리스트 추출
def get_pymysql_stock_list(conn, db_name):

    # 원하는 폴더의 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            result = [item[0] for item in cur.fetchall()]
            cur.close()

            return result
        
        
def get_sqlalchemy_investing_data(conn, investing_data_list):

    investing_df = pd.DataFrame()

    # 현재 DB 내 존재하는 테이블(종목) 추출
    for table in investing_data_list:
        conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
        sql = "SELECT * FROM investing_data.`{0}` ORDER BY 날짜 DESC LIMIT 1 ".format(table)

        result = conn.execute(sql)

        temp_df = pd.DataFrame(result.fetchall())
        investing_df = pd.concat([investing_df, temp_df], axis=1)
    
    investing_df['날짜'] = today
    return investing_df              


# stock df와 investing df를 병합
def get_sqlalchemy_stock_investing_merge_df_update(conn, stock_table_list, investing_df):
    
    complete_df = pd.DataFrame()
    investing_df['날짜'] = investing_df['날짜'].astype(str).astype(int) # 날짜 타입 int로 통일
    sql = "SELECT 전일대비, 상장주식수, 시가총액, 외국인현보유수량, 외국인현보유비율,\
        기관순매수량, 기관누적순매수량, 년, 월, 일 FROM stock_info.`{0}` WHERE 날짜={1} ORDER BY 시간 DESC LIMIT 1".format(code, yesterday)

    day_data = conn.execute(sql)
    day_stock_df = pd.DataFrame(day_data.fetchall())

    sql = "SELECT * FROM stock_info.`{0}` where  날짜 = {1}".format(code, today)
    table_data = conn.execute(sql)
    stock_df = pd.DataFrame(table_data.fetchall()) # DB내 테이블을 DF로 변환
    
    stock_df['날짜'] = stock_df['날짜'].astype(str).astype(int) # 날짜 타입 int로 통일
    day_col_list = ['전일대비','상장주식수','시가총액','외국인주문한도수량'
                ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
                ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량', '년', '월', '일'] 
    stock_df.drop(day_col_list,axis=1, inplace=True)
    
    stock_df = pd.concat([stock_df, day_stock_df], axis=1)
    
    complete_df = pd.concat([stock_df, investing_df],axis=1) # stock df 와 investing df 를 날짜 기준으로 merge
    complete_df = complete_df.fillna(method='ffill')
    conn.close()
    return complete_df     


In [7]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
investing_data_list = get_pymysql_stock_list(pymysql_conn, 'investing_data')
investing_df = get_sqlalchemy_investing_data(sqlalchemy_conn, investing_data_list)

In [1]:
stock_table_list_small = ['016790','095500','205470','278650','151860','053690','117580','063080','027710','011700'] # 소형주
stock_table_list_big = ['005930','373220','207940','000660','051910','247540','091990','066970','293490','028300'] # 대형주

In [ ]:
# 소형주 종목 별로 데이터 프레임 각각 생성
for code in stock_table_list_small:
    pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
    sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
    complete_df = get_sqlalchemy_stock_investing_merge_df_update(sqlalchemy_conn, code, investing_df)
    complete_df.to_pickle(f'../pickle/pickle_df/update/소형주_{today}_10개_update_{code}.pkl')

# 대형주 종목 별로 데이터 프레임 각각 생성
for code in stock_table_list_big:
    pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
    sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
    complete_df = get_sqlalchemy_stock_investing_merge_df_update(sqlalchemy_conn, code, investing_df)
    complete_df.to_pickle(f'../pickle/pickle_df/update/대형주_{today}_10개_update_{code}.pkl')

In [8]:
# 소형주 update 하나로
today = str(date.today() - timedelta(days=1)).replace('-','')
update_df = pd.DataFrame()
for code in stock_table_list_small:
    with open(f'../pickle/pickle_df/update/소형주_{today}_10개_update_{code}.pkl', 'rb') as f:
        stock_df = pickle.load(f)
    update_df = pd.concat([update_df, stock_df], axis=0)
    
update_df.to_pickle(f'../pickle/pickle_df/update/소형주_{today}_10개_update.pkl')

In [11]:
# 대형주 update 하나로
update_df = pd.DataFrame()
for code in stock_table_list_big:
    with open(f'../pickle/pickle_df/update/대형주_{today}_10개_update_{code}.pkl', 'rb') as f:
        stock_df = pickle.load(f)
    update_df = pd.concat([update_df, stock_df], axis=0)
    
update_df.to_pickle(f'../pickle/pickle_df/update/대형주_{today}_10개_update.pkl')